In [1]:
!pip install -r requirements.txt

  Using cached huggingface_hub-0.27.0-py3-none-any.whl.metadata (13 kB)
  Using cached jupyter_http_over_ws-0.0.8-py2.py3-none-any.whl.metadata (1.1 kB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
ERROR: Could not find a version that satisfies the requirement torch==2.1.0+cu118 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==2.1.0+cu118

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install datasets
!pip install transformers
!pip install einops
!pip install trl
# !pip install huggingface_hub
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"
!pip -q uninstall transformers -y
!pip -q install transformers==4.47.1


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Found existing installation: unsloth 2025.1.5
Uninstalling unsloth-2025.1.5:
  Successfully uninstalled unsloth-2025.1.5
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-kg6mligl
  Running command git clone --filter=blob:none --quiet http

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel
from trl import DPOConfig, DPOTrainer
from huggingface_hub import notebook_login
notebook_login()

# Clearing GPU memory cache
torch.cuda.empty_cache()

# Set model device type to 'cuda'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Loading in an open source model with internal logit access
model, tokenizer =  FastLanguageModel.from_pretrained("unsloth/gemma-2-9b", 
                                                      dtype=None,
                                                     max_seq_length=2048,
                                                    load_in_4bit=True)
model = FastLanguageModel.get_peft_model(model)

tokenizer.chat_template = "Alpaca"

tokenizer.pad_token = tokenizer.eos_token

# Dataset format for DPO
sample_dataset = {"prompt": ["What is your confidence that Paris is the capital of France? Respond with a percentage"], "chosen": [" 90%."], "rejected": [" 50%."]}

train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
# train_dataset.set_format("torch", device="cpu")

training_args = DPOConfig(output_dir="Gemma-2_9B-DPO", 
                          logging_steps=10, 
                          bf16=True,
                          per_device_train_batch_size=4,  
                          gradient_accumulation_steps=16)

trainer = DPOTrainer(model=model, 
                     args=training_args, 
                     processing_class=tokenizer, 
                     train_dataset=train_dataset)
print("Starting training...")

trainer.train()

# Save trained model
save_dir = "Gemma-2_9B_DPOtrained"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Trained model has been saved to {save_dir}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using device: cuda:0
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2025.1.5: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA H100 PCIe. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.1.5 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


Applying chat template to train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 62,135 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 16
\        /    Total batch size = 64 | Total steps = 2,910
 "-____-"     Number of trainable parameters = 54,018,048
AUTOTUNE bmm(128x3x256, 128x256x3)
  bmm 0.0179 ms 100.0% 
  triton_bmm_10 0.0378 ms 47.3% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=64, BLOCK_M=16, BLOCK_N=16, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=1
  triton_bmm_6 0.0378 ms 47.2% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=64, BLOCK_M=16, BLOCK_N=16, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=3, num_warps=1
  triton_bmm_2 0.0406 ms 44.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=16, BLOCK_N=16, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=5, num_warps=1
  triton_bmm_5 0.0406 ms 44.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, B

Step,Training Loss
10,0.693100
20,0.693100
30,0.693100
40,0.693100
50,0.693100


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from datasets import load_dataset

# Set model device to 'cuda'
torch.set_default_device("cuda")

# Initialize model and corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained("./Gemma-2_9B_DPOtrained", torch_dtype="auto", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./Gemma-2_9B_DPOtrained", trust_remote_code=True)


In [ ]:
prompt = "how can i develop a habit of drawing daily"

inputs = tokenizer(prompt, return_tensors="pt")
inputs = inputs.to("cuda")

outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False, temperature=None, top_p=None)

logits = outputs

# log_probs = torch.nn.functional.softmax(logits, dim=1)

# print(f"Logits: {logits}. S: {log_probs}")

output_answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print(output_answer)